# normal knob

* configfle(knob): normal 처리됨
* total_internal_df: 이상치 처리됨

### total_internal_df 이상치 처리 부분
internal과 knob과의 상관관계 개수는 상관이 없다고 할 수 있을까? 그러니까 상관계수를 비교하는데 각각의 데이터의 차이를 보여도 될까...?


In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
import xgboost
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from lightgbm import LGBMRegressor, plot_importance
from sklearn.metrics import roc_auc_score

In [2]:
# knob 종류 및 바운드
knobs_rdb = {
    
    "hash-max-ziplist-entries":     (256, 751), 
    "hash-max-ziplist-value": (16, 257),
    "activerehashing": ('yes', 'no'),  ##yes
    "hz": (1, 41),
    "dynamic-hz": ('yes', 'no'),

    "save0_0": (700,1100),
    "save0_1": (1,9),
    "save1_0": (100,500),
    "save1_1": (10,100),
    "save2_0": (30,90),
    "save2_1": (8000,12000),

    "rdbcompression": ('yes', 'no'), ##yes
    "rdbchecksum": ('yes', 'no'),    ##yes
    "rdb-save-incremental-fsync": ('yes', 'no'),   ## yes

    "activedefrag": ('yes','no'),  
    "active-defrag-threshold-lower": (1, 31),
    "active-defrag-threshold-upper": (70, 101),
    "active-defrag-cycle-min": (1, 31),
    "active-defrag-cycle-max": (70, 91),
    
    "maxmemory": (1000, 2900),   ##"volatile-lru"
    "maxmemory-policy":     ("volatile-lru", "allkeys-lru", "volatile-lfu", "allkeys-lfu", 
                             "volatile-random","allkeys-random", "volatile-ttl", "noeviction"),
    "maxmemory-samples": (3, 7),
    
    "lazyfree-lazy-eviction": ('yes', 'no'),
    "lazyfree-lazy-expire": ('yes', 'no'),
    "lazyfree-lazy-server-del": ('yes', 'no')
}

knobs_list = list(knobs_rdb.keys())
knobs_bound = list(knobs_rdb.values())

## 데이터 전처리
* 이상치 파악
* test, train 데이터 잘 나뉘어졌는지 확인

In [3]:
# config(knob) data
config_df = pd.read_csv("./data/result_config2_normal1.csv", sep=',')

# external(throughput) data
external_df = pd.read_csv("./data/external_metrics0.csv", sep=',')

In [4]:
# config(knog), external(throughput) 데이터 하나의 프레임으로 결합
dataset = config_df
dataset['throughput'] = external_df['Gets_KB/sec']

In [5]:
dataset.head(3)

,appendonly,appendfsync,auto-aof-rewrite-percentage,auto-aof-rewrite-min-size,no-appendfsync-on-rewrite,aof-rewrite-incremental-fsync,aof-use-rdb-preamble,save0_0,save0_1,save1_0,...,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz,throughput
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,910,5,176,...,5,no,no,no,496,126,yes,21,no,61645.41
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,997,4,306,...,5,yes,yes,yes,470,157,yes,31,yes,63270.32
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,945,4,244,...,5,no,yes,yes,376,169,no,25,yes,63602.49


In [6]:
# # 결측치 처리 
# dataset_nan = dataset.copy()
# # activedefrag 만
# subTotal_knobs_Active = dataset_nan.dropna()
# Active_index = list(subTotal_knobs_Active.index)  # 인덱스

In [7]:
# maxmemory knob 단위(mb) 처리
dataset['maxmemory']  = dataset['maxmemory'].str.strip('mb')
dataset['maxmemory'] = pd.to_numeric(dataset['maxmemory'])

In [8]:
# Object 변수 categorical 로 변경 (LightGBM에 바로 넣기 위해)
category_knob = []

for knob in dataset:
    if dataset[knob].dtype == 'O':  # Object인 경우 category 형으로 변경
        dataset[knob] = dataset[knob].astype('category')
        if knob in knobs_list:
            category_knob.append(knob)

In [9]:
# 설정한 knob들만 추출
extract_knobs_list = knobs_list.copy()
extract_knobs_list.append('throughput')  # target 값 추가

dataset = dataset[extract_knobs_list]

In [15]:
subTotal_knobs_Active = dataset.dropna()
Active_index = list(subTotal_knobs_Active.index)

In [21]:
dataset_Act = dataset.iloc[Active_index]
dataset_Act.drop(columns = ['throughput'], inplace=True)

/Users/joseong-un/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [22]:
dataset_Act.head(3)

,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz,save0_0,save0_1,save1_0,save1_1,save2_0,...,active-defrag-threshold-lower,active-defrag-threshold-upper,active-defrag-cycle-min,active-defrag-cycle-max,maxmemory,maxmemory-policy,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del
0,496,126,yes,21,no,910,5,176,45,51,...,16.0,85.0,20.0,75.0,1200,volatile-ttl,5,no,no,no
1,470,157,yes,31,yes,997,4,306,41,70,...,13.0,80.0,15.0,76.0,2100,volatile-ttl,5,yes,yes,yes
4,530,81,no,22,yes,913,5,265,40,68,...,20.0,86.0,19.0,80.0,2100,volatile-lfu,5,no,no,yes


## internal metrics

In [23]:
# cluster0 =[ 
# 'used_memory_rss',
# 'used_memory_rss_human',
# 'rss_overhead_ratio',
# 'rss_overhead_bytes',
# 'mem_fragmentation_ratio',
# 'mem_fragmentation_bytes']

# cluster1 = [
#     'uptime_in_seconds',
#     'hz',
#     'lru_clock',
#     'used_memory_peak',
#     'maxmemory',
#     'maxmemory_human',
#     'rdb_last_save_time',
#     'total_net_output_bytes',
#     'used_cpu_sys',
#     'used_cpu_user'
# ]

# cluster2 = [
#     'allocator_allocated',
#     'allocator_active',
#     'allocator_frag_bytes',
#     'instantaneous_ops_per_sec',
#     'instantaneous_input_kbps',
#     'instantaneous_output_kbps'
# ]

# cluster3 = [
#     'allocator_resident',
#     'allocator_rss_ratio',
#     'allocator_rss_bytes'  
# ]

cluster0 =[ 
    'allocator_allocated',
    'allocator_frag_bytes',
    'uptime_in_seconds',
    'used_memory_peak',
    'configured_hz',
    'used_cpu_sys',
    'maxmemory',
    'hz',
    'used_cpu_user',
    'maxmemory_human'
]

cluster1 = [
    'allocator_rss_bytes',
    'allocator_resident',
    'allocator_rss_ratio'
]

cluster2 = [
    'used_memory_rss',
    'rss_overhead_bytes',
    'mem_fragmentation_bytes',
    'used_memory_rss_human'
]


remove_knob = [
    'configured_hz', 'total_net_output_bytes', 'keyspace_hits', 'keyspace_misses'
]

knob_list = [
    "save0_0",
    "save0_1",
    "save1_0",
    "save1_1",
    "save2_0",
    "save2_1",

    "rdbcompression",
    "rdbchecksum",
    "rdb-save-incremental-fsync",

    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
    
    "maxmemory",
    
    "maxmemory-policy",
    "maxmemory-samples",
    "lazyfree-lazy-eviction",
    "lazyfree-lazy-expire",
    "lazyfree-lazy-server-del",

    "hash-max-ziplist-entries", 
    "hash-max-ziplist-value",
    "activerehashing",
    "hz",
    "dynamic-hz"
]

activedefrag_knobs = [
    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
]

In [24]:
# 데이터 불러오기 internal metrics와 knob들의 상관관계분석
internal_csv = pd.read_csv('./data/internal_metrcis1.csv')

In [25]:
internal_csv
internal_csv = internal_csv.iloc[Active_index] 

In [29]:
# 컬럼 추출 함수
def extract_data(cluster_list, df):
    extracted_df = pd.DataFrame()
    for elem in cluster_list:
        extracted_df[elem] = df[elem]

    return extracted_df    

In [64]:
# 클러스터 합치기
# cluster = cluster0+cluster1+cluster2+cluster3+remove_knob
cluster = cluster2

In [65]:
# 클러스터 요소에 해당되는 metrics 25개 추출
total_internal_df = extract_data(cluster, internal_csv)

In [66]:
# throughput 결측치 제거
total_internal_df.dropna()

,used_memory_rss,rss_overhead_bytes,mem_fragmentation_bytes,used_memory_rss_human
0,42033152,-176128,3410544,40.09
1,41979904,-229376,3275392,40.04
4,42053632,-155648,3431024,40.11
6,42037248,-2273280,3414640,40.09
7,42029056,-180224,3406448,40.08
8,42020864,-188416,3398256,40.07
9,42016768,-192512,3394160,40.07
10,41779200,-380928,3129088,39.84
11,42024960,-184320,3402352,40.08
12,42033152,-176128,3410544,40.09


In [67]:
total_internal_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 0 to 99
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   used_memory_rss          58 non-null     int64  
 1   rss_overhead_bytes       58 non-null     int64  
 2   mem_fragmentation_bytes  58 non-null     int64  
 3   used_memory_rss_human    58 non-null     float64
dtypes: float64(1), int64(3)
memory usage: 4.3 KB


In [68]:
def get_outlier(df=None, column=None, weight=1.5):
    
    # target 값과 상관관계가 높은 열을 우선적으로 진행
    quantile_25 = np.percentile(df[column].values, 25)
    quantile_75 = np.percentile(df[column].values, 75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight

    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight

    outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
    return outlier_idx

# 함수 사용해서 이상치 값 삭제
# outlier_idx = get_outlier(df=total_internal_df, column='used_memory_rss', weight=1.5)

# internal = total_internal_df.drop(outlier_idx, axis=0)
# knob = dataset.drop(outlier_idx, axis=0)

In [69]:
numerical_var= ['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [70]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau

internal_exit_pv = []

def get_corr_withoutOutlier(internal_m, knob):
    nan_corr_var = ['rss_overhead_ratio', 'mem_fragmentation_ratio', 'used_memory_peak','total_net_output_bytes', 'allocator_active', 'allocator_resident', 'allocator_rss_ratio', 'allocator_rss_bytes']
    
    if internal_m in nan_corr_var:
        corr_result = pearsonr(total_internal_df[internal_m], dataset_Act[knob])
        if corr_result[1] <=0.05:
            internal_exit_pv.append(internal_m)
            return corr_result[0]
#             print(corr_result[0])
        return 0
    
    # 이상치 인덱스 계산
    outlier_idx = get_outlier(df=total_internal_df, column=internal_m, weight=1.5)  # outlier의 인덱스를 받는다. 
    
    internal = total_internal_df.drop(outlier_idx, axis=0)
    knobdata = dataset_Act.drop(outlier_idx, axis=0)
    
    corr_result = pearsonr(internal[internal_m], knobdata[knob])
    if corr_result[1] <=0.05:
        internal_exit_pv.append(internal_m)
#         print(corr_result[0])
        return corr_result[0]
    return 0



In [71]:
numeric = ['hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz',
 'save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max',
 'maxmemory',
 'maxmemory-samples',
 ]

In [74]:
# # 결과 데이터 프레임 생성
# # corr_result = pd.DataFrame(index = numeric)
# corr_result['cluster2'] = cluster_k_corr_var

In [75]:
# corr_result

,cluster0,cluster1,cluster2
hash-max-ziplist-entries,0.000000,0.000000,0.082325
hash-max-ziplist-value,0.000000,-0.310415,-0.157867
hz,0.237633,0.000000,-0.075925
save0_0,0.026633,0.000000,-0.238859
save0_1,0.000000,0.000000,0.000000
save1_0,0.000000,0.000000,0.000000
save1_1,0.000000,0.000000,0.000000
save2_0,-0.035834,0.000000,0.144744
save2_1,-0.030911,0.000000,0.000000
active-defrag-threshold-lower,0.000000,0.000000,0.000000


In [72]:
# knob_li = knob.columns 
# knob_li.remove('throughput')

import numpy as np

# cluster_k_corr_var = []

for knob in dataset_Act[numeric]:
    print('######{}######'.format(knob))
    result_corrs = []
    for internal in total_internal_df:
        print(internal)
        result_corrs.append(get_corr_withoutOutlier(internal, knob)) 
    num_result = np.array(result_corrs)
    cluster_k_corr_var.append(np.mean(num_result).tolist()) 
        
        
        

######hash-max-ziplist-entries######
used_memory_rss
rss_overhead_bytes
mem_fragmentation_bytes
used_memory_rss_human
######hash-max-ziplist-value######
used_memory_rss
rss_overhead_bytes
mem_fragmentation_bytes
used_memory_rss_human
######hz######
used_memory_rss
rss_overhead_bytes
mem_fragmentation_bytes
used_memory_rss_human
######save0_0######
used_memory_rss
rss_overhead_bytes
mem_fragmentation_bytes
used_memory_rss_human
######save0_1######
used_memory_rss
rss_overhead_bytes
mem_fragmentation_bytes
used_memory_rss_human
######save1_0######
used_memory_rss
rss_overhead_bytes
mem_fragmentation_bytes
used_memory_rss_human
######save1_1######
used_memory_rss
rss_overhead_bytes
mem_fragmentation_bytes
used_memory_rss_human
######save2_0######
used_memory_rss
rss_overhead_bytes
mem_fragmentation_bytes
used_memory_rss_human
######save2_1######
used_memory_rss
rss_overhead_bytes
mem_fragmentation_bytes
used_memory_rss_human
######active-defrag-threshold-lower######
used_memory_rss
rss_ov

In [73]:
cluster_k_corr_var

[0.0823247458108301,
 -0.1578674062040607,
 -0.07592454316525034,
 -0.23885883954275108,
 0.0,
 0.0,
 0.0,
 0.14474360944464038,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.24045157260321456,
 0.0]

In [60]:
internal_exit_pv = list(set(internal_exit_pv))

In [61]:
internal_exit_pv

['allocator_rss_ratio', 'allocator_resident', 'allocator_rss_bytes']

In [38]:
len(internal_exit_pv)

17